In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(hosts=['http://localhost:9200/'])

In [47]:
import pandas as pd
data=pd.read_csv('books_info.csv')

In [48]:
data=data.fillna(value='nan')

In [49]:
data.columns

Index(['title', 'url', 'contributors', 'date', 'format', 'fulltext_url',
       'trove_id', 'language', 'rights', 'pages', 'form', 'volume', 'parent',
       'children', 'text_downloaded', 'text_file'],
      dtype='object')

In [50]:
data.shape

(42174, 16)

In [51]:
data.isna().sum()

title              0
url                0
contributors       0
date               0
format             0
fulltext_url       0
trove_id           0
language           0
rights             0
pages              0
form               0
volume             0
parent             0
children           0
text_downloaded    0
text_file          0
dtype: int64

In [52]:
data_f=data.to_dict('record')

/tmp/ipykernel_3663/1305562343.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  data_f=data.to_dict('record')


In [8]:
data_f[0]

{'title': 'Goliath Joe, fisherman / by Charles Thackeray (Wobbegong)',
 'url': 'https://trove.nla.gov.au/work/10013347',
 'contributors': 'Thackeray, Charles',
 'date': '1900-1919',
 'format': 'Book|Book/Illustrated',
 'fulltext_url': 'https://nla.gov.au/nla.obj-2831231419',
 'trove_id': 'nla.obj-2831231419',
 'language': 'English',
 'rights': 'Out of Copyright|http://rightsstatements.org/vocab/NKC/1.0/',
 'pages': 130,
 'form': 'Book',
 'volume': 'nan',
 'parent': 'nan',
 'children': 'nan',
 'text_downloaded': True,
 'text_file': 'goliath-joe-fisherman-by-charles-thackeray-wob-nla.obj-2831231419.txt'}

In [89]:


mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "url": {"type": "text"},
            "contributors": {"type": "keyword"},
            "date":{"type":"keyword"},
            "format": {"type": "text"},
            "fulltext_url": {"type": "keyword"},
            "trove_id": {"type": "text"},
            "language": {"type": "keyword"},
            "rights": {"type": "keyword"},
            "pages": {"type": "integer"},
            "form": {"type": "keyword"},
            "volume": {"type": "keyword"},
            "parent": {"type": "keyword"},
            "children": {"type": "keyword"},
            "text_downloaded": {"type": "keyword"},
            "text_file": {"type": "keyword"},


            
        }
    }
}

index_name = "books"

es.indices.create(index=index_name, body=mapping)


/tmp/ipykernel_3663/3760057039.py:32: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body=mapping)
/tmp/ipykernel_3663/3760057039.py:32: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=index_name, body=mapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'books'})

In [90]:
i=1
for row in data_f:
    es.index(index='books', body=row)

/tmp/ipykernel_3663/862392675.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='books', body=row)
/tmp/ipykernel_3663/862392675.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='books', body=row)


In [91]:

res = es.count(index="books", body={"query": 
                                     {"bool": 
                                      {"must_not":
                                      {"term":
                                      {"children":"nan"}}}}})



/tmp/ipykernel_3663/3714474380.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.count(index="books", body={"query":
/tmp/ipykernel_3663/3714474380.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.count(index="books", body={"query":


In [92]:
res['count']

240

In [56]:

res = es.count(index="books", body={"query": 
                                     {"bool": 
                                      {"must_not":
                                      {"term":
                                      {"parent":"nan"}}}}})



/tmp/ipykernel_3663/2721874177.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.count(index="books", body={"query":
/tmp/ipykernel_3663/2721874177.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.count(index="books", body={"query":


In [57]:
res['count']

1423

In [58]:

res = es.search(index="books", body={
                                    "query": 
                                     {"bool": 
                                      {"must_not":
                                      {"term":
                                      {"parent":"nan"}}}},
                                       'aggs':
                                         {'avg_pages':
                                        {'avg':
                                        { "field":"pages"
                                            
                                        }}}})


/tmp/ipykernel_3663/85668804.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="books", body={
/tmp/ipykernel_3663/85668804.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="books", body={


In [59]:
res['aggregations']['avg_pages']['value']

298.4525650035137

In [60]:

res = es.search(index="books", body={
                                    "query": 
                                     {"bool": 
                                      {"must_not":
                                      {"term":
                                      {"children":"nan"}}}},
                                       'aggs':
                                         {'avg_pages':
                                        {'avg':
                                        { "field":"pages"
                                            
                                        }}}})

/tmp/ipykernel_3663/428908489.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="books", body={
/tmp/ipykernel_3663/428908489.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="books", body={


In [61]:
res['aggregations']['avg_pages']['value']

0.0

In [94]:
from elasticsearch import Elasticsearch


index_name = "books"

# Define the terms aggregation
aggs = {
    "group_by_field": {
        "terms": {
            "field": "date"
        }
    }
}

# Search for documents and apply the aggregation
body = {
    "size": 0,
    "aggs": aggs
}

res = es.search(index=index_name, body=body)

# Extract the aggregation results
buckets = res["aggregations"]["group_by_field"]["buckets"]

# Print the counts for each value in columns
for bucket in buckets:
    value = bucket["key"]
    count = bucket["doc_count"]
    print(f"{value}: {count}")

2018: 1594
2017: 1378
2016: 1085
1900: 777
2015: 770
2019: 725
1716: 606
2012: 598
2014: 503
2013: 493


/tmp/ipykernel_3663/848199082.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index=index_name, body=body)
/tmp/ipykernel_3663/848199082.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index=index_name, body=body)


In [2]:
from cassandra.cluster import Cluster
clstr=Cluster()
session=clstr.connect()


In [42]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS ath 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
    session.set_keyspace('ath')
except Exception as e:
    print(e)

In [29]:

index_name = 'books'


search_query = {
    "bool": {
        "must_not": {
            "term": {
                "parent": "nan"
            }
        }
    }
}

result = es.search(index=index_name, query=search_query,size=2000)


/home/farid/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [39]:
trove_id=[]
title=[]
url=[]
contributors=[]
fulltext_url=[]
book_format=[]
pages=[]
date=[]
for i in result['hits']['hits']:
        trove_id.append(i['_source']['trove_id']) 
        title.append(i['_source']['title'])
        url.append(i['_source']['url'])
        contributors.append(i['_source']['contributors'])
        fulltext_url.append( i['_source']['fulltext_url'])
        book_format.append( i['_source']['format'])
        pages.append(int(i['_source']['pages']))
        if i['_source']['date'].split('-')[0] == 'nan':
            date.append(0)
        else:
            date.append(int(i['_source']['date'].split('-')[0]))





In [43]:
create_table_query = """
CREATE TABLE parent (
  trove_id TEXT,
  title TEXT,
  url TEXT,
  contributors TEXT,
  fulltext_url TEXT,
  book_format TEXT,
  pages INT,
  date INT,
  PRIMARY KEY (trove_id)
);
"""
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)   

In [44]:
for i in range(len(trove_id)):
    session.execute("INSERT INTO parent (trove_id, title, url,contributors,fulltext_url,book_format,pages,date) VALUES (%s, %s,%s,%s, %s,%s,%s,%s)", 
                    (trove_id[i],title[i],url[i],contributors[i],fulltext_url[i],book_format[i],pages[i],date[i]))


In [52]:
result_count = session.execute("SELECT COUNT(pages) AS count , AVG(pages) AS avg FROM parent")
for row in result_count:
    print(row)

Row(count=1419, avg=298)


In [50]:
result_year = session.execute("SELECT * FROM parent WHERE date >= %s AND date <= %s  ALLOW FILTERING", [1, 2000])
for row in result_year:
    print(row)

Row(trove_id='nla.obj-56155448', book_format='Book', contributors='nan', date=1716, fulltext_url='http://nla.gov.au/nla.obj-56155448', pages=600, title='Kangxi zi dian', url='https://trove.nla.gov.au/work/237411458')
Row(trove_id='nla.obj-56180948', book_format='Book', contributors='nan', date=1716, fulltext_url='http://nla.gov.au/nla.obj-56180948', pages=600, title='Kangxi zi dian', url='https://trove.nla.gov.au/work/237411458')
Row(trove_id='nla.obj-57729010', book_format='Book', contributors='nan', date=1800, fulltext_url='http://nla.gov.au/nla.obj-57729010', pages=64, title='Zhong xiao jie yi er du mei zhuan', url='https://trove.nla.gov.au/work/12608633')
Row(trove_id='nla.obj-56186140', book_format='Book', contributors='nan', date=1716, fulltext_url='http://nla.gov.au/nla.obj-56186140', pages=600, title='Kangxi zi dian', url='https://trove.nla.gov.au/work/237411458')
Row(trove_id='nla.obj-56155644', book_format='Book', contributors='nan', date=1716, fulltext_url='http://nla.gov.au

In [53]:


index_name = 'books'


search_query = {
    "bool": {
        "must_not": {
            "term": {
                "children": "nan"
            }
        }
    }
}

result = es.search(index=index_name, query=search_query,size=2000)

/home/farid/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [54]:
trove_id_ch=[]
title_ch=[]
url_ch=[]
contributors_ch=[]
fulltext_url_ch=[]
book_format_ch=[]
pages_ch=[]
date_ch=[]
for i in result['hits']['hits']:
        trove_id_ch.append(i['_source']['trove_id']) 
        title_ch.append(i['_source']['title'])
        url_ch.append(i['_source']['url'])
        contributors_ch.append(i['_source']['contributors'])
        fulltext_url_ch.append( i['_source']['fulltext_url'])
        book_format_ch.append( i['_source']['format'])
        pages_ch.append(int(i['_source']['pages']))
        if i['_source']['date'].split('-')[0] == 'nan':
            date_ch.append(0)
        else:
            date_ch.append(int(i['_source']['date'].split('-')[0]))





In [57]:
create_table_query = """
CREATE TABLE children (
  trove_id TEXT,
  title TEXT,
  url TEXT,
  contributors TEXT,
  fulltext_url TEXT,
  book_format TEXT,
  pages INT,
  date INT,
  PRIMARY KEY (trove_id)
);
"""
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)   

In [58]:
for i in range(len(trove_id_ch)):
    session.execute("INSERT INTO children (trove_id, title, url,contributors,fulltext_url,book_format,pages,date) VALUES (%s, %s,%s,%s, %s,%s,%s,%s)", 
                    (trove_id_ch[i],title_ch[i],url_ch[i],contributors_ch[i],fulltext_url_ch[i],book_format_ch[i],pages_ch[i],date_ch[i]))


In [59]:
result_count = session.execute("SELECT COUNT(pages) AS count , AVG(pages) AS avg FROM children")
for row in result_count:
    print(row)

Row(count=238, avg=0)


In [60]:
result_year = session.execute("SELECT * FROM children WHERE date >= %s AND date <= %s  ALLOW FILTERING", [1, 2000])
for row in result_year:
    print(row)

Row(trove_id='nla.obj-33236945', book_format='Book|Book/Illustrated', contributors='Society for Nautical Research (London, England)', date=1935, fulltext_url='http://nla.gov.au/nla.obj-33236945', pages=0, title='Lists of men-of-war, 1650-1700', url='https://trove.nla.gov.au/work/13097671')
Row(trove_id='nla.obj-39267340', book_format='Book', contributors='Campbell, Bessie, 1870-1964', date=1889, fulltext_url='https://nla.gov.au/nla.obj-39267340', pages=0, title='[Bessie Campbell, Banjo Queen : a collection of music programmes, ephemera relating to her career]', url='https://trove.nla.gov.au/work/237278882')
Row(trove_id='nla.obj-475664029', book_format='Government publication|Book', contributors='nan', date=1860, fulltext_url='http://nla.gov.au/nla.obj-475664029', pages=0, title='Report of the proceedings of the Queensland Government schooner "Spitfire" in search of the mouth of the River Burdekin, on the north-eastern coast of Australia : and of the exploration of a portion of that co

In [41]:
session.execute("DROP KEYSPACE ath")
clstr.shutdown()

In [84]:
es.indices.delete(index='books')


/tmp/ipykernel_3663/3647000100.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index='books')


ObjectApiResponse({'acknowledged': True})